# Assignment 4

# Question 1

The answer is yes, because K-means is to put each data points into different cluster based on the data point's distance to the cluster center. For Gaussian Mixture it use probability for each data point, so if the Gaussian Mixture have large variance, points in Gasussian Mixture still might be assigned to the same cluster which is same in K-means.

# Question 2

(a): (1)A and C blocked in the condition of B and D is true, because we can see from graph that is head-to-tail both, the path from A to C meet at B and D, and B,D is in set B,D
     (2)B and D blocked in the condition of A and C is false, because we can see from graph that is tail-to tail for path B->D meet at A, and A is in set A. for set C is follow the head-to-head, and C shoud not in the set of C
     
(b): (1)A and C blocked in the condition of B and D is false, because we can see from graph that is head-to-head both, so the node B and D should not in the set B and D
     (2)B and D blocked in the condition of A and C is true, because we can see from graph that is tail-to tail both, so the node A and C is in the set A and C

# Question 3

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

In [2]:
dataset = np.array([[4.6,2.9],[4.7,3.2],[4.9,3.1],[5.0,3.0],[5.1,3.8],[5.5,4.2],[5.9,3.2],[6.0,3.0],[6.2,2.8],[6.7,3.1]])
red = np.array([6.2,3.2])
blue = np.array([6.5,3.0])
green = np.array([6.6,3.7])
centroid = np.array([[6.2,3.2],[6.5,3.0],[6.6,3.7]])

This is to calculate the Euclidean Distance follow the PDF

In [3]:
def euclidean_distance(p1,p2):
    return np.sqrt(np.sum((p1-p2)**2))

This is to check if the centroid of cluster is converage, and i use and error like 0.00001 to test

In [4]:
def converage(centroids1,centroids2,error):
    return (centroids1 - centroids2).any() < error 

Kmeans algorithm:
      1. First i need to have dataset and inital cluster center, in this question, we already have the center, so i don't need to random create it
      2. while the cluster is not converage, for each data point i will calculate the euclidean distance between current data point and each cluster center, choose the minimal one, and update the label of this data point
      3. once the cluster is converaged, print the result

In [5]:
def k_means(data,centroid):
    iterations = 0
    centroids = centroid
    is_converaged = False
    distance_in_new_centroids = np.zeros(data.shape[0])
    labels = np.zeros(data.shape[0])
    
    
    while not is_converaged:
        iterations += 1 
        previous_centroids = np.copy(centroids)
        for i in range(data.shape[0]):
            min_distance = np.inf
            min_index = -1
            for j in range(len(centroid)):
                distance = euclidean_distance(data[i],centroids[j])
                if distance < min_distance:
                    min_distance = distance
                    min_index = j
                    labels[i] = j
            label = labels[i]
            distance_in_new_centroids = euclidean_distance(data[i],centroids[np.int(label)])**2
            
        for z in range(len(centroid)):
            centroids[z] = np.mean(data[labels == z],axis = 0)
        is_converaged = converage(previous_centroids,centroids,0.00001)
        print("For iteration %d" % iterations)
        print("Centroids is",centroids)
        print("cluster scatter",np.sum(distance_in_new_centroids))
        
    return centroids,labels,iterations

In [6]:
k_means(dataset,centroid)

For iteration 1
Centroids is [[5.17142857 3.17142857]
 [6.45       2.95      ]
 [5.5        4.2       ]]
cluster scatter 0.050000000000000086
For iteration 2
Centroids is [[4.8   3.05 ]
 [6.2   3.025]
 [5.3   4.   ]]
cluster scatter 0.08499999999999998
For iteration 3
Centroids is [[4.8   3.05 ]
 [6.2   3.025]
 [5.3   4.   ]]
cluster scatter 0.2556250000000001


(array([[4.8  , 3.05 ],
        [6.2  , 3.025],
        [5.3  , 4.   ]]), array([0., 0., 0., 0., 2., 2., 1., 1., 1., 1.]), 3)

# (a)

The center of red cluster after one interation is [5.17142857,3.17142857]

# (b)

The center of green cluster after two iteration is [5.3,4.0]

# (c)
The center of blue cluster when clustering converages is [6.2,3.025]

# (d)

We can see from my algorithm that the clusters need 2 iterations to converage

# Question 4

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import multivariate_normal

Get the data from the txt file, and store in numpy array

In [2]:
data = np.loadtxt('C:\\Users\\WesleyZhou\\Desktop\\CS559\\Assignment4\\multigauss.txt')

Put the dataset into np.matrix type

In [3]:
data = np.matrix(data,copy = True)

# (a)
Define the expection method, which is the a question, and follow the lecture pdf, we can see that expection is to calculate each model's gamma

In [4]:
def expection(X,k,pi,mu,cov):
    #number of data points
    n = X.shape[0]
    # defin the gamma matrix which rows represent number of data points, columns represent gamma
    gamma = np.mat(np.zeros((n,k)))
    # First i need to calculate the probility of each data points in each models
    prob = np.zeros((n,k))
    for i in range(k):
        min_eig = np.min(np.real(np.linalg.eigvals(cov[i])))
        if min_eig < 0:
            cov[i] -= 10*min_eig * np.eye(*cov[i].shape)
        prob[:,i] = multivariate_normal.pdf(X,mu[i],cov[i])
    prob = np.mat(prob)
    
    # And then i need to calculate each data points' gamma and return the gamma
    for j in range(k):
        gamma[:,j] = pi[j] * prob[:,j]
    for z in range(n):
        gamma[z,:] /= np.sum(gamma[z,:])
    return gamma

# (b)
Define the Maximization of Means,return new maximization of mean

In [5]:
def MaximizeMean(X,k,w):
    # define the new mu
    mu = np.zeros((k, X.shape[1]))
    # update each model
    for i in range(k):
        #calculate the new nk
        nk = np.sum(w[:,i])
        #for each features calculate the mean value, and store in the mu array
        for j in range(X.shape[1]):
            mu[i,j] = np.sum(np.multiply(w[:,i],X[:,j]))/nk
    return mu

# (c)
Define the Maximization of Covariances, return new covariances

In [6]:
def MaximizeCovariance(X,k,w,mu):
    #Define the new cov array
    cov = []
    #update each models
    for i in range(k):
        # calculate the nk
        nk = np.sum(w[:,i])
        # define the cov matrix for model i
        cov_new = np.mat(np.zeros((X.shape[1],X.shape[1])))
        #calculate the calculate the cov matrix and store in cov array
        for j in range(X.shape[0]):
            cov_new += w[j,i] * (X[j] - mu[i]).T * (X[j] - mu[i]) /nk
        cov.append(cov_new)
    cov = np.array(cov)
    return cov
        

# (d)
Define the Maximization of Mixture Weights， return new weight for each gaussian

In [7]:
def MaximizeMixtures(k,w):
    #Define the new pi
    pi =  np.zeros(k)
    #update for each model
    for i in range(k):
        # Follow the equation of lecture pdf,we can calculate the new pi for each model, and store in pi array
        nk = np.sum(w[:,i])
        pi[i] = nk/w.shape[0]
    return pi

# (e)
Define the EM method, and use the log likelihood to check if it is incresing for each iteration of EM or not

In [8]:
def em(X,k,pi0,mu0,cov0,nIter):
    # init for each parameters
    pi = pi0
    mu = mu0
    cov = cov0
    # for each iteration
    for i in range(nIter):
        #First use the E step, to get the gamma
        gamma = expection(X,k,pi,mu,cov)
        #Then use the M step to maximize pi,mu,cov
        pi = MaximizeMixtures(k,gamma)
        mu = MaximizeMean(X,k,gamma)
        cov = MaximizeCovariance(X,k,gamma,mu0)
        print("Iteration:" + " ", i)
        # Use the log likelihood to check if it is incresing or not
        result = 0
        for n in range(X.shape[0]):
            result_sub = 0
            for kloop in range(k):
                distr = multivariate_normal(mu[kloop],cov[kloop])
                result_sub += (pi[kloop]+0.00001)+ distr.pdf(X[n,:])
            result += np.log(result_sub)
        print(result)
    
    return mu,cov,pi

Init parameters, such as X,k,pi0,mu0,cov0,nIter 

In [9]:
X = data
k = 5
pi0 = np.array([1.0 / 5] * 5)
mu0 = np.random.rand(5, X.shape[1])
cov0 = np.array([np.eye(X.shape[1])]*5)
nIter = 15

# Conclusion
Get the result from EM method, and we can see that the log likelihood is increasing for each iteration

In [10]:
em(X,k,pi0,mu0,cov0,nIter)
# expection(X,k,pi0,mu0,cov0)

Iteration:  0
49.387055228482254
Iteration:  1
50.50509066123417
Iteration:  2
51.37606837915545
Iteration:  3
52.326484535834915
Iteration:  4
53.51415870463857
Iteration:  5
54.97094471404937
Iteration:  6
56.78393795178275
Iteration:  7
59.22828335765747
Iteration:  8
61.91409567177046
Iteration:  9
63.771183052353756
Iteration:  10
64.85683043597305
Iteration:  11
65.51701144785633
Iteration:  12
65.9205405479128
Iteration:  13
66.1945743100717
Iteration:  14
66.43460208284296


(array([[ 1.46174325, -1.51949314],
        [ 1.70588951,  1.98436295],
        [-1.12202722, -1.12883922],
        [-1.13950689, -1.15712251],
        [-0.69663996,  0.57971282]]), array([[[ 5.24119189, -3.76934984],
         [-3.76934984,  9.60878772]],
 
        [[ 4.12263572,  4.61912725],
         [ 4.61912725,  6.58878813]],
 
        [[10.74395456,  9.35783412],
         [ 9.35783412,  9.8945444 ]],
 
        [[12.09017953,  8.8373457 ],
         [ 8.8373457 ,  7.43375999]],
 
        [[ 9.50510033, -6.03698054],
         [-6.03698054,  7.37495646]]]), array([0.14537466, 0.1568149 , 0.10677199, 0.19030021, 0.40073823]))